### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras import backend
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Versioning
print(keras.__version__)
print(tf.__version__)

2.2.4
1.10.0


### Read in Normalized Option Pricing Data Set

In [3]:
option_df = pd.read_pickle("opt_data_z.pkl")
## Move data into numpy array
prices = option_df.values

In [4]:
# X values
X = prices[:,:4]

# Y values 
y = prices[:,4:5]

# Use Sklearn library to randomly split data into training/testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Build Keras/Scikit Learn Function

In [5]:
def create_model(optimizer = 'rmsprop'):

    nodes = 200

    # Create Model
    model = Sequential()

    # Layer 1
    model.add(Dense(nodes, input_shape=(4,))) 
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    # Layer 2
    model.add(Dense(nodes)) 
    model.add(Activation ('relu'))
    model.add(Dropout(0.25))

    # Layer 3
    model.add(Dense(nodes))
    model.add(Activation ('relu'))
    model.add(Dropout(0.25))

    # Layer 4 - Output Layer
    model.add(Dense(1))
    model.add(Activation('elu'))
    
    model.compile(loss='mse', optimizer = optimizer, metrics=['accuracy'])
    
    return model

#### Instantiate Keras Classifier

In [6]:
neural_network = KerasClassifier(build_fn = create_model, verbose = 0)

#### Define Search Parameters 

In [10]:
## Small list of possible paramaters - goal is to explore search framework rather than all possible architectures

epochs = [2]
#batches = [5]
optimizers = ['rmsprop', 'adam']

hyperparameters = dict(optimizer=optimizers, epochs=epochs)

#### Set stopping criteria 

In [11]:
callbacks_list = [EarlyStopping(monitor = 'loss', patience = 1)]

In [12]:
# Create grid search
grid = GridSearchCV(estimator = neural_network, param_grid = hyperparameters, cv = 2)

# Fit grid search
grid_result = grid.fit(X_train, y_train, verbose = 2, callbacks = callbacks_list)

Epoch 1/2
 - 88s - loss: 239275280.2732 - acc: 3.0779e-05
Epoch 2/2
 - 90s - loss: 86708103.7942 - acc: 4.0012e-05
Epoch 1/2
 - 90s - loss: 229946885.5718 - acc: 4.3090e-05
Epoch 2/2
 - 91s - loss: 83134501.2185 - acc: 4.9246e-05
Epoch 1/2
 - 105s - loss: 265803697.9286 - acc: 2.7701e-05
Epoch 2/2
 - 110s - loss: 175359988.1144 - acc: 2.4623e-05
Epoch 1/2
 - 110s - loss: 221100237.4126 - acc: 4.3090e-05
Epoch 2/2
 - 107s - loss: 91608149.1495 - acc: 6.1557e-05
Epoch 1/2
 - 191s - loss: 326712278.6832 - acc: 3.3857e-05
Epoch 2/2
 - 182s - loss: 193910085.5300 - acc: 4.0012e-05


In [13]:
grid_result.best_params_

{'epochs': 2, 'optimizer': 'rmsprop'}